In [1]:
import warnings

import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import pyodbc
import turbodbc
from turbodbc import connect


import config
import config_media_costs
from normalize_funcs import normalize_columns_types
from create_dicts import get_cleaning_dict
from db_funcs import createDBTable, downloadTableToDB, get_mssql_table


db_name = config.db_name

In [22]:
# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем ширину столбцов по контенту
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

warnings.simplefilter(action='ignore', category=FutureWarning)
# убираем лишние предупреждения
pd.set_option('mode.chained_assignment', None)

sep_str = '*' * 50

In [3]:
def get_adex_dicts(dict_name, search_lst=None):
    search_lst_str = ''
    # если передан список уникальных ИД объявлений, то преобразуем список в строку
    if search_lst:
        # достаем из настроек словарей навзание колонк,по которой нужно выполнить фильтрацию в исходной БД Медиаскоп
        search_col_name = config_media_costs.adex_ad_dicts[dict_name][3]
        
        search_lst_str = config.get_lst_to_str(search_lst)
        # формируем условие для фильтрации запроса к БД
        search_lst_str = f'where {search_col_name} in ({search_lst_str})'
    # для каждого справочника формируем строку с указанием полей для загрузки
    if 'tv_Ad'==dict_name:
        cols = 'vid, name as adName, notes, atid, stdur, alid, blid, sblid, mlid, slid2, slid3, slid4, fiss, slaid, slvid'
        

    # отправляем запрос в БД Медиа инвестиции
    query = f'select {cols} from {dict_name} {search_lst_str}' 
    df = get_mssql_table(config.investments_db_name, query=query, conn_lst=config.conn_lst)
    
    # переименовываем поля - приводим их в соответсвии с названиями из ТВ Индекс
    # перебираем справочник config_tv_investments.rename_cols_dict
    # если название поля из Медиа инвестиции есть в ключах, то забираем пару ключ-значение
    # чтобы передать для присвоения нового названия
    new_cols_name = {key: value for (key, value) in config_media_costs.rename_cols_dict.items() if key in list(df.columns)}
    df = df.rename(columns=new_cols_name)
    
    return df

In [24]:
# создаем, которая забирает из справочников объявлений Медиаскоп adId и их характеристики
# только только для ИД, которые есть в нашей Базе
# на вход принимаем тип  медиа
# название таблицы-справочника в Медиаскоп, по которому нужно получить данные
def get_ads_adex_dict(media_type):
    adex_dict_name = media_type + '_Ad'
    # формируем запрос к таблице Фактов
    query = f"select distinct adId  from media_{media_type}_costs"
    ad_id_dict = get_mssql_table(config.db_name, query=query)
    # приводим датаФрейм с adId к списку
    search_lst = list(ad_id_dict['adId'])
    # отправляем запрос в Медиаскоп, чтобы получить данные из справочника
    df = get_adex_dicts(adex_dict_nam, search_lst)
    
    return df

tv_Ad
Все ок. Подключились!
Загрузка завершена успешно


In [23]:
media_type = 'tv'
merge_cleaning_adex_ad_dict(df, media_type)

C:\Users\o.bogomolov\Desktop\Jupyter_notebook\42_02_mediascope_costs_n_ratings\create_dicts.py:68: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(full_cleaning_link, header=None, usecols=cols_count)



**************************************************
Перезаписываем справочник: adex_ad_dict_list_tv  в БД: mediascope_x5

Все ок. Подключились!
Таблица: adex_ad_dict_list_tv успешно удалена в БД: mediascope_x5
##########
Все ок. Подключились!
Пустая таблица adex_ad_dict_list_tv успешно создана в БД mediascope_x5
Все ок. Подключились!
Скрипт запущен 2025-04-10 10:57:42.203903
Данные добавлены в БД: mediascope_x5, таблица: adex_ad_dict_list_tv
Скрипт отработал 2025-04-10 10:57:42.381558
Время выполнения задачи: 0:00:00.177655
Загрузка завершена. Данные успешно добавлены в БД: mediascope_x5
##################################################



In [17]:
media_type

'TV'

In [10]:
df.shape

(4735, 15)

In [11]:
df.head(2)

,adId,adName,adNotes,ad_type_id,adStandardDuration,advertiserListId,brandListId,subbrandListId,modelListId,articleList2Id,articleList3Id,articleList4Id,fiss,adSloganAudioId,adSloganVideoId
0,2089789,,,5,0,1231528,2855444,2566612,3740869,50079,144958,267476,20131020,1,1
1,2493941,,,5,0,314040,578484,519062,2193612,50079,144958,267476,20150316,1,1


In [ ]:
# for key, value in config_media_costs.adex_dicts.items():
#     print(key)

In [18]:
# функция для обонвления основного справочнка объявлений ad_dict по каждому типу медиа в отдельности
#  справочники ad_dict содержат ИД с List
# ее запускаем в самую последнюю очередь заливки данных по расходам за 1 месяц
# НО ПЕРЕД заливкой новых объявлений в гугл докс

def merge_cleaning_adex_ad_dict(df, media_type):
    # забираем гугл докс с чисткой
    df_cleaning_dict = get_cleaning_dict(media_type=media_type)
    # нормализуем типы данных
    df_cleaning_dict = normalize_columns_types(df_cleaning_dict, config.custom_ad_dict_int_lst) 
    
    # создаем список из названий полей, которые нам нужны дальше для метчинга
    custom_cols_list = [col[:col.find(' ')] for col in config.custom_ad_dict_vars_list]
    custom_cols_list = list(set(custom_cols_list) - set(['ad_id', 'media_type']))
    # оставляем только нужные поля из справочника чистки
    df_cleaning_dict = df_cleaning_dict[custom_cols_list]

    
# обрабатываем ответ из справочника Медиаскоп List
    media_type = media_type.upper()
# добавляем тип медиа и создаем ключ для объединения с таблице чистки
    df['media_type'] = media_type
    df['media_key_id'] = df['media_type'] + '_' + df['adId'].astype('str')
    # Привдим первую дату выхода к формату yyyy-mm-dd
    df['fiss'] = df['fiss'].astype('str')
    df['adFirstIssueDate'] = df['fiss'].apply(lambda x: x[:4] + '-' + x[4:6] + '-' + x[6:] if x!='' else x)
    # объединяем справочник Медиаскоп List с таблицей Чистки гугл докс
    df = df.merge(df_cleaning_dict, how='left', left_on=['media_key_id'], right_on=['media_key_id'])
    # если в справочнике Медиаскоп List НЕ нашлось сопоставления с гугл докс чисткой
    # значит - это новое объвление и ставим ему флаг 2
    df['cleaning_flag'] = df['cleaning_flag'].fillna(2)

    # у нас есть словарь config_media_costs.adex_ad_lst_dicts, в котором находятся все необходимые параметры 
    # забираем из файла create_dicts - словарь, где 
    # ключ - это название таблицы в Медиаскоп (для дальнейшего удобства так сделано)
    # значение - это список, который содержит:
    # [0] - название таблицы в БД
    # [1] - список полей с типами данных для БД
    # [2] - список полей с целочисленными значениями для нормализации
    # [3] - поле, по которому выполняется фильтрация в исходной БД Медиаскоп при ОБНОВЛЕНИИ справочника
    
    # здесь формируем ключ, чтобы достать из словаря нужные значения
    table_name = f'{media_type.lower()}_Ad'
    
    # [0] - название таблицы в БД
    dict_table_name = config_media_costs.adex_ad_lst_dicts[table_name][0]
    # [1] - список полей с типами данных для БД
    vars_cols_lst = config_media_costs.adex_ad_lst_dicts[table_name][1]
    # [2] - список полей с целочисленными значениями для нормализации
    int_cols_lst = config_media_costs.adex_ad_lst_dicts[table_name][2]
    
    # убираем не нужный текст после пробела, например brand_main nvarchar(150) -> brand_main
    cols_lst = [i[:i.find(' ')] for i in vars_cols_lst]
    # оставляем в датаФрейме только нужные поля
    df = df[cols_lst]
    # приводим типы данных в порядок
    df = normalize_columns_types(df, int_cols_lst)
    

    print()
    print(sep_str)
    print(f'Перезаписываем справочник: {dict_table_name}  в БД: {config.db_name}')
    print()
    # чистим справочник перед сохранением новых данных
    createDBTable(config.db_name, dict_table_name, vars_cols_lst, flag='drop')
    # записываем справочник с обновленными ИД и данными из гугл чистки
    downloadTableToDB(config.db_name, dict_table_name, df)